In [1]:
import json
import requests
from src import utils
import time
import os
from pprint import pprint
import random
from tqdm import tqdm

In [2]:
api_path = '/userhomes/philhoon/kt-ai-challenge/api-zs.json'

In [3]:
api = utils.read_json_file(api_path)

In [4]:
# output_path = '/userhomes/philhoon/kt-ai-challenge/result'

In [5]:
# os.makedirs(output_path, exist_ok=True)
# output_file_name = 'result-' + input_file.split('/')[-1]
# output_file = output_path + '/' + output_file_name
# print(f'output_file : {output_file}')

In [6]:
# peft tunning
# peft1 = '/userhomes/philhoon/kt-ai-challenge/result/kor-gsm8k-peft1-test.jsonl'
# peft2 = '/userhomes/philhoon/kt-ai-challenge/result/kor-gsm8k-peft2-test.jsonl'

# ICL
# zs = '/userhomes/philhoon/kt-ai-challenge/result/zero-shot-public_mwp_data_v2_preprocess'
# zs_cot = '/userhomes/philhoon/kt-ai-challenge/result/zero-shot-cot-public_mwp_data_v2_preprocess'
# one_shot_cot = '/userhomes/philhoon/kt-ai-challenge/result/one-shot-cot-result-ko-gsm8k-test.jsonl'


In [7]:
class model_api():
    def __init__(self, url, headers):
        self.url = url
        self.headers = headers
        
    def generate(self, body):
        
        try:
            response = requests.post(self.url, data=body, headers=self.headers, verify=False)
        
        except:
            print("retrying due to an error......")
            time.sleep(20)
            return self.generate(body)
        
        return response
        
        

In [8]:
model = model_api(api['url'], api['headers'])
BODY_TEMP = api['body']

In [9]:
def get_body(prompt, body_temp):
    body_temp["utterance"] = prompt
    return json.dumps(body_temp)

In [10]:
def get_gsm8k_zs3(que):
    prompt = f'다음 질문에 차근차근 답하시오. {que}'
    return prompt

In [11]:
## Test dataset

# GSM8K 8792 (7473/1319)
gsm8k = '/userhomes/philhoon/kt-ai-challenge/data/openai-gpt-3.5-turbo-0301-2048-0.0-KOR-gsm8k-test.jsonl'

# MWP_KR_DATA 3750
mwp_kr = '/userhomes/philhoon/kt-ai-challenge/data/public_mwp_data_v2_reasoning-test.jsonl'

# AddSub 395
addsub = '/userhomes/philhoon/kt-ai-challenge/eng_data/kor-D_addsub-test.json'

# MultiArith 600
multiarith = '/userhomes/philhoon/kt-ai-challenge/eng_data/kor-D_multiarith-test.json'

# SingleEq 395
single_eq = '/userhomes/philhoon/kt-ai-challenge/eng_data/kor-D_single_eq-test.json'

In [12]:
gsm8k_data = utils.read_jsonlines(gsm8k)
mwp_kr_data = utils.read_jsonlines(mwp_kr)
addsub_data = utils.read_jsonlines(addsub)
multiarith_data = utils.read_jsonlines(multiarith)
single_eq_data = utils.read_jsonlines(single_eq)

In [13]:
print(len(gsm8k_data))
print(len(mwp_kr_data))
print(len(addsub_data))
print(len(multiarith_data))
print(len(single_eq_data))

1319
720
79
120
102


In [14]:
# random_indices

In [15]:
data = single_eq_data

In [16]:
data[0]

{'sample_index': 1,
 'completion_index': 0,
 'question': 'There were 28 bales of hay in the barn. Tim stacked more bales in the barn today. There are now 54 bales of hay in the barn. How many bales did he store in the barn ?',
 'answer': '26.0',
 'reasoning_prompt': "Q: There were 28 bales of hay in the barn. Tim stacked more bales in the barn today. There are now 54 bales of hay in the barn. How many bales did he store in the barn ?\nA: Let's think step by step.",
 'reasoning_completion': ' \n\nThere were 28 bales of hay in the barn. \n\nThis is our starting point, and we are told that Tim stacked more bales in the barn today. \n\nThere are now 54 bales of hay in the barn. \n\nThis is our ending point. We are trying to find out how many bales Tim stacked in the barn, so this is the information that we need. \n\nTo find out how many bales Tim stacked in the barn, we need to find the difference between the starting point and the ending point. \n\nThe difference between 28 and 54 is',
 '

In [17]:
for ins in tqdm(data):
    kor_que = ins['kor_question']
    prompt = get_gsm8k_zs3(kor_que)
    body = get_body(kor_que, BODY_TEMP)
    
#     kor_que = ins['question']
#     body = get_body(kor_que, BODY_TEMP)
    
    response = model.generate(body)
    res_context = response.json()['data'][0]['result'][0]

#     ins['zs_infer'] = res_context
    print(f"prompt : \n{prompt}")
    print(f"response : \n{res_context}")
    print(f"----------------------------")
#     print(f"reason : \n{ins['reason']}")
    print(f"answer : \n{ins['answer']}")
    print(f"=============================")
#     jsonl_writer.write_json_line(ins)

  0%|                                                                                                         | 0/102 [00:00<?, ?it/s]/userhomes/philhoon/miniconda3/envs/kt/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
  1%|▉                                                                                                | 1/102 [00:01<02:28,  1.47s/it]

prompt : 
다음 질문에 차근차근 답하시오. 헛간에는 28개의 건초 더미가 있었습니다. 팀은 오늘 헛간에 더 많은 건초 더미를 쌓았습니다. 이제 헛간에는 54개의 건초 더미가 있습니다. 그는 헛간에 몇 개의 건초 더미를 저장했을까요?
response : 
그는 헛간에 54개의 건초 더미를 저장했습니다.
----------------------------
answer : 
26.0


/userhomes/philhoon/miniconda3/envs/kt/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
  2%|█▉                                                                                               | 2/102 [00:09<08:52,  5.33s/it]

prompt : 
다음 질문에 차근차근 답하시오. 레스토랑에서 식사를 한 후, 샐리, 샘, 앨리사는 계산서를 균등하게 나누기로 결정했습니다. 각각의 사람이 45달러를 지불했다면, 총 계산서는 얼마인가요?
response : 
샐리, 샘, 앨리사가 45달러를 지불했으므로, 총 계산서는 45 * $45 = $<<45*45=45>>45입니다.각 사람이 계산서를 균등하게 나누기로 결정했으므로, 총 계산서의 분배는 다음과 같습니다:샐리, 샘, 앨리사 (45달러) = $<<45*45=45>>45샘샘샘 앨리사 (45달러) = $<<45*45=45>>45앨리사따라서, 총 계산서는 45 * 45 = $<<45*45=45>>45입니다.
----------------------------
answer : 
135.0


/userhomes/philhoon/miniconda3/envs/kt/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
  3%|██▊                                                                                              | 3/102 [00:12<07:23,  4.48s/it]

...

prompt : 
다음 질문에 차근차근 답하시오. 힐트 여사는 책상에서 물 분수까지의 거리를 측정했습니다. 그 거리는 30 피트였습니다. 만약 그녀가 오늘 물 분수에 4번 가게 된다면, 힐트 여사는 총 몇 피트를 걸을까요?
response : 
힐트 여사는 오늘 물 분수에 4번 가게 됩니다. 따라서 총 4 * 30 = 180 피트를 걸게 됩니다.
----------------------------
answer : 
120.0


/userhomes/philhoon/miniconda3/envs/kt/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
 99%|██████████████████████████████████████████████████████████████████████████████████████████████ | 101/102 [05:42<00:03,  3.36s/it]

prompt : 
다음 질문에 차근차근 답하시오. 루시는 212마리의 물고기를 가지고 있습니다. 루시가 280마리의 물고기를 가지려면 얼마나 더 많은 물고기를 사야합니까?
response : 
루시가 280마리의 물고기를 가지려면 212마리의 물고기를 가지고 있으므로, 루시는 212 * 212 = 감시할 물고기가 280마리가 있어야 합니다.루시가 280마리의 물고기를 가지려면 212 * 280 = 감시할 물고기가 280마리가 있어야 합니다.따라서 루시는 280마리의 물고기를 가지려면 212마리 * 280 = 감시할 물고기가 280마리가 있어야 합니다.
----------------------------
answer : 
68.0


/userhomes/philhoon/miniconda3/envs/kt/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
100%|███████████████████████████████████████████████████████████████████████████████████████████████| 102/102 [05:44<00:00,  3.37s/it]

prompt : 
다음 질문에 차근차근 답하시오. 마누엘은 177달러의 초기 예금으로 저축 계좌를 개설했습니다. 그가 다음 19주 동안 500달러를 저축하려면 매주 얼마를 저축해야 합니까?
response : 
마누엘은 다음 19주 동안 500달러를 저축하려면 매주 50달러를 저축해야 합니다.
----------------------------
answer : 
17.0


In [ ]:
zs_cot_data = utils.read_jsonlines(zs_cot)
zs_data = utils.read_jsonlines(zs)

In [ ]:
print(len(zs_cot_data))
len(zs_cot_data) == len(zs_data)

In [ ]:
from pprint import pprint

In [ ]:
cnt = 0
for ins in zs_data:
#     pprint(ins)
    print(f"question :\n{ins['question']}")
#     print(f"zs_cot_infer :\n{ins['zs_cot_infer']}")
    print(f"zs_infer :\n{ins['zs_infer']}")
    print(f"answer :\n{ins['answer']}")
    print("===========================")
    cnt += 1
    if cnt == 50:
        break